<a href="https://colab.research.google.com/github/Jeady1565/A.I-Projects/blob/main/Deep_Convolutional_Q_Learning_for_Pac_Man_Complete_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [15]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip-install pip ale-py               #installed ale_py package
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.
/bin/bash: line 1: pip-install: command not found


### Importing the libraries

In [2]:
import os
import ale_py   #For ALE/Pacman v5 support
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):

  def __init__(self, action_size, seed = 42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.conv1 = nn.Conv2d(3, 32, kernel_size = 8, stride = 4)        #Brains and Eyes of CNN L1
    self.bn1 = nn.BatchNorm2d(32)                                     #Batch for Stabalized training same channel at CNN L1 one to the other
    self.conv2 = nn.Conv2d(32, 64, kernel_size = 4, stride = 2)
    self.bn2 = nn.BatchNorm2d(64)
    self.conv3 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1)
    self.bn3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1)
    self.bn4 = nn.BatchNorm2d(128)                                    #CNN  Layer Output
    self.fc1 = nn.Linear(10 * 10 * 128, 512)                          #Full connections
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, action_size)

  def forward(self, state):                                 #Forward Propigate the signals from the images to the CNN to corresponding batch one to the other with retifier application function
    x = F.relu(self.bn1(self.conv1(state)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))
    x = x.view(x.size(0), -1)                               #CNN Flattening Tensor while keep first batch dimension
    x = F.relu(self.fc1(x))                                 #Full Connection to output layer
    x = F.relu(self.fc2(x))
    return self.fc3(x)                                      #Output layer

## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
import ale_py                                               #installed ale_py for pacman support

env = gym.make('ALE/MsPacman', full_action_space = False)   #Using newer verstion of Pacman :\
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (210, 160, 3)
State size:  210
Number of actions:  9


/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:525: UserWarning: WARN: Using the latest versioned environment `ALE/MsPacman-v5` instead of the unversioned environment `ALE/MsPacman`.
  logger.warn(


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4     #hyperparameters from Lunar code
minibatch_size = 64      #reduced minibatch size
discount_factor = 0.99   #Same Gamma

### Preprocessing the frames

In [6]:
from PIL import Image
from torchvision import transforms

def preprocess_frame(frame):                  #Transforming to NP array/tensors
  frame = Image.fromarray(frame)
  preprocess = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])  #transform resize to 128. 128 and convert to py tensor
  return preprocess(frame).unsqueeze(0)  #to keep batch argument as 1st dimension

### Implementing the DCQN class

In [7]:
class Agent():                                                                             #Acting Agent

  def __init__(self, action_size):                                                         #pasted code from lunar code
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")           #state sized and self state size and t step removed
    self.action_size = action_size
    self.local_qnetwork = Network(action_size).to(self.device)                             #soft update removed from code. Not needed
    self.target_qnetwork = Network(action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = deque(maxlen = 10000)

  def step(self, state, action, reward, next_state, done):
    state = preprocess_frame(state)
    next_state = preprocess_frame(next_state)
    self.memory.append((state, action, reward, next_state, done))                       #to process state and add to next memory code
    if len(self.memory) > minibatch_size:
      experiences = random.sample(self.memory, k = minibatch_size)                      #sampling from minibatch size
      self.learn(experiences, discount_factor)                                           #learn from experiences satge

  def act(self, state, epsilon = 0.):                                                   #pasted code from lunar code changing state to frame funce instead of experince relay
    state = preprocess_frame(state).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):                                           #Seperating next states which will unzip and stack states
    states, actions, rewards, next_states, dones = zip(*experiences)
    states = torch.from_numpy(np.vstack(states)).float().to(self.device)
    actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

### Initializing the DCQN agent

In [8]:
agent = Agent(number_actions)

### Training the DCQN agent

In [9]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 10000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: 256.10
Episode 200	Average Score: 307.40
Episode 300	Average Score: 481.60
Episode 400	Average Score: 418.90
Episode 500	Average Score: 384.20
Episode 588	Average Score: 500.50
Environment solved in 488 episodes!	Average Score: 500.50


## Part 3 - Visualizing the results

In [16]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder
!pip install imageio[ffmpeg]

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30, format='FFMPEG')

show_video_of_model(agent, 'ALE/MsPacman')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 51.6 MB/s eta 0:00:00
